## 라이브러리 가져오기

In [22]:
import pandas as pd
import numpy as np
import folium
from folium import plugins

## 데이터 불러오기

In [59]:
# 약수터 데이터셋 - 출처: https://www.data.go.kr/data/15017324/standard.do
yaksuteo = pd.read_csv("https://www.data.go.kr/download/15017324/standard.do?dataType=csv", encoding="EUCKR")
yaksuteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   약수터명      1472 non-null   object 
 1   소재지도로명주소  306 non-null    object 
 2   소재지지번주소   1462 non-null   object 
 3   위도        1315 non-null   float64
 4   경도        1316 non-null   float64
 5   지정일자      648 non-null    object 
 6   일평균이용인구수  1472 non-null   int64  
 7   수질검사일자    1472 non-null   object 
 8   수질검사결과구분  1472 non-null   object 
 9   부적합항목     603 non-null    object 
 10  관리기관전화번호  1472 non-null   object 
 11  관리기관명     1472 non-null   object 
 12  데이터기준일자   1472 non-null   object 
 13  제공기관코드    1472 non-null   int64  
 14  제공기관명     1472 non-null   object 
dtypes: float64(2), int64(2), object(11)
memory usage: 172.6+ KB


In [60]:
yaksuteo[yaksuteo["약수터명"]=="형제"]

,약수터명,소재지도로명주소,소재지지번주소,위도,경도,지정일자,일평균이용인구수,수질검사일자,수질검사결과구분,부적합항목,관리기관전화번호,관리기관명,데이터기준일자,제공기관코드,제공기관명
904,형제,NaN,경기도 구리시 교문동 산89-1,37.591802,127.122297,1996-03-23,1450,2018-04-24,적합,없음,031-550-8587,경기도 구리시청,2020-09-15,3980000,경기도 구리시


## 데이터 정제하기

In [27]:
# 시각화 하고자 하는 데이터만 추출하기
yaksuteo = yaksuteo.loc[:, ["약수터명", "위도", "경도", "일평균이용인구수", "수질검사결과구분"]]

In [28]:
# 비어있는 값 제거하기
yaksuteo = yaksuteo.dropna()
yaksuteo.isnull().sum()

약수터명        0
위도          0
경도          0
일평균이용인구수    0
수질검사결과구분    0
dtype: int64

## 지도 만들기 (1/2)
### 포함되는 데이터
- 약수터 이름 및 일평균 이용 인구
- 위도 경도
- 수질 통과 여부

In [39]:
# 서울 위도 경도
yaksu_map = folium.Map(location=[37.5665, 126.9780])

In [40]:
name_and_popularity = yaksuteo["약수터명"] + ", 일평균" + yaksuteo["일평균이용인구수"].astype(str) + "명 사용"
name_and_popularity = name_and_popularity.to_numpy()
name_and_popularity

array(['귀성약수터, 일평균300명 사용', '감로천약수터, 일평균100명 사용', '불로약수터, 일평균100명 사용',
       ..., '용화약수터, 일평균240명 사용', '수암약수터, 일평균300명 사용', '성덕사, 일평균300명 사용'],
      dtype=object)

In [41]:
# 위도와 경도를 묶어서 리스트로 변환해주기
geocodes = np.array(list(zip(yaksuteo["위도"], yaksuteo["경도"])))
geocodes

array([[ 37.46227725, 126.772928  ],
       [ 37.468825  , 126.732545  ],
       [ 37.46883023, 126.7242531 ],
       ...,
       [ 37.364565  , 126.889788  ],
       [ 37.372041  , 126.897731  ],
       [ 37.604114  , 127.085838  ]])

In [42]:
# 수질검사가 적합일 경우 True, 부적합인 경우 False
yaksu_quality = [quality == "적합" for quality in yaksuteo["수질검사결과구분"]]

# 결과에 따라 아이콘을 연두색 또는 빨간색으로 표시
yaksu_icon = [folium.Icon(color="green", icon = "info_sign") if quality else folium.Icon(color="red", icon = "info_sign") for quality in yaksu_quality]

In [43]:
# 지도 그리고 보여주기
plugins.MarkerCluster(geocodes, popups=name_and_popularity, icons=yaksu_icon).add_to(yaksu_map)
yaksu_map

Output hidden; open in https://colab.research.google.com to view.

## 약수 인기도 Heatmap으로 표현하기

In [57]:
how_much_hotae = np.array(list(zip(yaksuteo["위도"], yaksuteo["경도"], yaksuteo["일평균이용인구수"])))
yaksuteo[yaksuteo['일평균이용인구수'] == yaksuteo["일평균이용인구수"].max()]

,약수터명,위도,경도,일평균이용인구수,수질검사결과구분
904,형제,37.591802,127.122297,1450,적합


In [97]:
hotae_hotae = folium.Map(location=[37.5665, 126.9780])
plugins.HeatMap(how_much_hotae, radius=10, min_opacity = 0.8, max_val=1450,).add_to(hotae_hotae)
hotae_hotae

In [96]:
temp = folium.Map(location=[37.5665, 126.9780])
plugins.HeatMap([[37.5665, 126.9780, 100]], radius=10, min_opacity = 0.8,).add_to(temp)
temp